# 实验目标：实现归一化的自定义层：

   - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
   - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
      $$
      \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
      $$
      其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
   - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。

# 一、实现自定义层

当继承Layer类时，我们可以实现如下方法

- \_\_init__()

    自定义层的属性，并且可以不依赖输入数据的形状通过使用`add_weight()`来创建层的权重
- build(self, input_shape)

    此方法可以依赖输出的形状来创建权重，使用`add_weight()`。通过调用`build()`，TensorFlow会自动调用`__call__`方法来创建层

-  call(self, *args, **kwargs)

    确保`build()`被调用之后再调用`__call__`，`call()`执行的是将层应用到输入参数上的逻辑，其中有两个保留的关键字参数：一个是`training`，表示是否在训练模式被调用，另一个是`mask`，它是一个布尔类型的张量，主要用于序列数据的编码，例如一句话中的一个词、一段音频的一帧等等

-  get_config(self)

    返回一个字典，此字典包含用于初始化层的配置信息，如果字典中的键与`__init__()`中的参数不同，则会重写`from_config(self)`方法。当存储此层或者一个模型包含此层时，这个方法才会被使用

In [1]:
import tensorflow as tf

class CustomLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        self.alpha = self.add_weight(
            name="alpha",
            shape=input_shape[-1:],
            dtype=tf.float32,
            initializer="ones"
        )
        self.beta = self.add_weight(
            name="beta",
            shape=input_shape[-1:],
            dtype=tf.float32,
            initializer="zeros"
        )

    def call(self, inputs, *args, **kwargs):
        mu, var = tf.nn.moments(inputs, axes=-1, keepdims=True)       # 对每一行求均值和方差
        epsilon = 1e-6
        sigma = tf.sqrt(var)
        norm = (inputs - mu) / (sigma + epsilon)
        result = self.alpha * norm + self.beta
        return result


# 二、验证自定义层的准确性

In [2]:
X = tf.random.uniform(shape=[30, 20], minval=1, maxval=20, dtype=tf.float32)

In [3]:
my_layer = CustomLayer()
normalization_layer = tf.keras.layers.LayerNormalization()

In [4]:
import numpy as np
data1 = my_layer(X)
data2 = normalization_layer(X)
np.allclose(data1, data2)

False

结果运行出来是`False`

我回看了类的实现，应该没有问题，下面就把两个归一化的结果打印出来看一下，发现数据相似，所以我改了np.allclose里面的容差，之后再调用，就为`True`了

In [5]:
data1

<tf.Tensor: shape=(30, 20), dtype=float32, numpy=
array([[-1.2945998e+00, -1.0137023e+00,  1.4716278e+00,  1.1307096e+00,
         3.0066076e-01, -5.5665612e-01, -2.4310045e-01, -9.1936356e-01,
         1.6836731e+00,  2.6917112e-01, -5.0492245e-01, -8.8905156e-02,
         1.2880772e+00,  1.5148846e+00, -1.4296892e-01, -1.1292329e+00,
        -5.6265187e-01, -9.2318416e-01, -1.3480707e+00,  1.0685523e+00],
       [-6.6033208e-01, -9.0552151e-01,  1.4658487e+00, -9.3400216e-01,
        -8.5886694e-02, -8.1067634e-01,  1.2776144e+00, -1.2200834e+00,
         1.7703592e+00, -1.1621585e+00, -1.0133742e+00,  7.7794552e-01,
         1.5690403e-01, -1.5173006e-01,  1.5554997e+00,  1.1225269e+00,
        -5.1372093e-01,  5.8529907e-01, -3.5711300e-02, -1.2188036e+00],
       [-1.0596192e+00, -1.1423843e+00,  1.9604258e-02,  8.2243717e-01,
         5.5187857e-01,  1.3003840e+00,  1.4731011e+00, -5.3532958e-01,
         1.7592604e-01, -1.5608615e+00, -1.6229913e+00, -1.7625725e-01,
         7.1

In [6]:
data2

<tf.Tensor: shape=(30, 20), dtype=float32, numpy=
array([[-1.29458165e+00, -1.01368809e+00,  1.47160769e+00,
         1.13069415e+00,  3.00656766e-01, -5.56648314e-01,
        -2.43096918e-01, -9.19350684e-01,  1.68365002e+00,
         2.69167542e-01, -5.04915297e-01, -8.89037699e-02,
         1.28805959e+00,  1.51486373e+00, -1.42966792e-01,
        -1.12921715e+00, -5.62643945e-01, -9.23171222e-01,
        -1.34805179e+00,  1.06853771e+00],
       [-6.60322785e-01, -9.05508876e-01,  1.46582854e+00,
        -9.33989167e-01, -8.58853459e-02, -8.10665011e-01,
         1.27759695e+00, -1.22006643e+00,  1.77033484e+00,
        -1.16214228e+00, -1.01336002e+00,  7.77934968e-01,
         1.56902015e-01, -1.51727811e-01,  1.55547833e+00,
         1.12251163e+00, -5.13713658e-01,  5.85291147e-01,
        -3.57106477e-02, -1.21878672e+00],
       [-1.05959821e+00, -1.14236164e+00,  1.96038689e-02,
         8.22420835e-01,  5.51867604e-01,  1.30035830e+00,
         1.47307181e+00, -5.35318911e-

In [7]:
np.allclose(data1, data2, rtol=1.e-4, atol=1.e-8)

True

上述判断两个矩阵值相似是通过NumPy函数评估的，总觉得不严谨，而且还改了容差的默认值

所以，是不是可以对两个样本进行相似性度量来确定实现是否正确，核技巧（Kernel Trick）？显然不好实现。emmm... 相似性度量？是基于向量之间的距离得出的，两向量距离越近，相似性越高

那么，对于两个值的比较来说，其差值越接近于0，越相似。于是，联想到了MSE（均方误差），对于数据中的每一列来说，可以将其视为一个回归问题的目标，比如第1列数据是某地第1周的房价，第2列数据是某地第2周的房价等等，因此，对于上述两个矩阵来说，可以将其分别视为y_pred与y_true，只不过对多个目标的回归（例如对每周的房价进行回归）

下面，回到问题本身，对于两个矩阵比较值相似来说，没必要区分是每行看成一个回归指标还是每列看成一个回归指标，并且，还有一个问题就是，不管是行还是列，对其算MSE之后，肯定是相应的序列，所以，我们可以将其归约为一个指标，这里使用tf.reduce_mean,只要两个矩阵的值足够相似，归约后的平均值就会越接近于0

In [8]:
mse = tf.keras.losses.MeanSquaredError()
tf.reduce_mean(mse(data1, data2))

<tf.Tensor: shape=(), dtype=float32, numpy=2.9045247e-10>

可以看到，结果是一个接近于0的数，所以，实现正确！！！